In [ ]:
# Run this code to analyse the whole database of videos and process it

### 1)Imports and setup

In [40]:
import os
import base64
import aiohttp
import asyncio
import json
import imageio
import re
import time
from PIL import Image
import numpy as np
import colorsys
import aiofiles
import nest_asyncio
from tqdm.asyncio import tqdm
from dotenv import load_dotenv
from scenedetect import VideoManager, SceneManager
from scenedetect.detectors import ContentDetector
import logging
from datetime import datetime

# Configure logging
logging.basicConfig(level=logging.INFO)

# Load OpenAI API key from .env file
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

# Apply nest_asyncio to handle the running event loop
nest_asyncio.apply()

# Concurrency limit
semaphore = asyncio.Semaphore(5)

# A dictionary to store characters across frames
character_frames = {}


### 2) Video Analysis Functions

In [41]:
def analyze_video(video_path, threshold=27.0):
    if not os.path.exists(video_path):
        raise FileNotFoundError(f"The video file {video_path} does not exist.")
    
    video_manager = VideoManager([video_path])
    scene_manager = SceneManager()
    scene_manager.add_detector(ContentDetector(threshold=threshold))

    video_manager.set_downscale_factor()
    video_manager.start()

    scene_manager.detect_scenes(frame_source=video_manager)
    scene_list = scene_manager.get_scene_list()

    video_manager.release()

    logging.info(f'Detected {len(scene_list)} scenes:')
    for i, scene in enumerate(scene_list):
        logging.info(f'Scene {i + 1}: Start {scene[0].get_timecode()} / Frame {scene[0].get_frames()}, '
              f'End {scene[1].get_timecode()} / Frame {scene[1].get_frames()}')

    return scene_list


### 3) Frame Extraction Function

In [42]:
def extract_frames_imageio(video_path, scenes, output_dir):
    reader = imageio.get_reader(video_path)
    for i, scene in enumerate(scenes):
        start_frame, end_frame = scene
        
        # Convert FrameTimecode to integer frame numbers
        start_frame_num = int(start_frame)
        end_frame_num = int(end_frame)
        
        # Calculate the middle frame of the scene
        middle_frame = (start_frame_num + end_frame_num) // 2
        
        # Set the reader to the middle frame and extract it
        reader.set_image_index(middle_frame)
        frame = reader.get_next_data()
        
        # Save the frame as an image
        output_path = os.path.join(output_dir, f'scene_{i + 1}.jpg')
        imageio.imwrite(output_path, frame)
        print(f"Extracted and saved middle frame of scene {i + 1} as {output_path}", flush=True)


### 4) Image Processing Function

In [43]:
async def encode_image(image_path):
    async with aiofiles.open(image_path, "rb") as image_file:
        content = await image_file.read()
        return base64.b64encode(content).decode('utf-8')

def get_color_category(color):
    r, g, b = [x / 255.0 for x in color]
    h, l, s = colorsys.rgb_to_hls(r, g, b)

    primary_hues = {
        "red": (0.0, 0.1),  
        "yellow": (0.1, 0.18),
        "green": (0.25, 0.4),
        "blue": (0.55, 0.75),
    }

    for color_name, hue_range in primary_hues.items():
        if hue_range[0] <= h <= hue_range[1]:
            return color_name

    if (l >= 0.9 and s <= 0.1):
        return "white"
    if (l <= 0.1 and s <= 0.1):
        return "black"

    return "non-primary"

def analyze_image_colors(image_path):
    image = Image.open(image_path)
    image = image.convert('RGB')
    data = np.array(image)

    unique_colors, counts = np.unique(data.reshape(-1, data.shape[2]), axis=0, return_counts=True)
    total_pixels = int(counts.sum())

    color_counts = {
        "Red": 0,
        "Yellow": 0,
        "Green": 0,
        "Blue": 0,
        "White": 0,
        "Black": 0,
        "Non-primary": 0
    }

    for color, count in zip(unique_colors, counts):
        category = get_color_category(tuple(color))
        color_counts[category.capitalize()] += int(count)

    color_percentages = {color: (count / total_pixels) * 100 for color, count in color_counts.items()}
    primary_total = color_counts["Red"] + color_counts["Yellow"] + color_counts["Blue"]
    color_dominance = "Primary colors" if primary_total > color_counts["Non-primary"] else "Non-primary colors"

    return {
        "Color Analysis": {
            "Colors Found": {
                color: {
                    "Pixel Count": count,
                    "Percentage": f"{color_percentages[color]:.2f}%"
                } for color, count in color_counts.items()
            },
            "Dominance": color_dominance
        }
    }


### 5) OpenAI API Interaction

In [62]:
async def send_image_to_openai(image_path, base64_image, retries=3):
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }

    payload = {
        # "model": "gpt-4o-mini",
        # "model": "gpt-3.5-turbo",
        "model": "gpt-4o",
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": """
                        Analyze the following image and provide a detailed description in the format of JSON only. Ensure the output is strictly in JSON format without any additional text or code block formatting. The JSON should include the following standardized labels:
                        1. **Image Analysis**: The root dictionary containing all analysis data.
                        2. **Suitability**: Indicators of suitability for general audiences.
                        3. **Objects**: List and analysis of objects detected in the image.
                        4. **Place**: Information about the place or setting in the image.
                        5. **Characters**: Information about any characters or people in the image.
                        """
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/png;base64,{base64_image}"
                        }
                    }
                ]
            }
        ],
        "max_tokens": 750
    }

    for attempt in range(retries):
        try:
            async with aiohttp.ClientSession() as session:
                async with session.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload) as response:
                    # Log the status code and full response for debugging
                    status = response.status
                    response_text = await response.text()
                    
                    print(f"Response Status Code: {status}")
                    print(f"Response Content: {response_text}")

                    if status == 429:
                        print("Rate limit exceeded, retrying...")
                        await asyncio.sleep(2 ** attempt)
                        continue
                    elif status == 200:
                        content = await response.json()
                        
                        # Log the full JSON content
                        print(f"Full JSON Response for {image_path}: {content}")
                        
                        if 'choices' in content:
                            message_content = content['choices'][0].get('message', {}).get('content', '').strip()
                            try:
                                return json.loads(message_content)
                            except json.JSONDecodeError as e:
                                print(f"Error decoding JSON from OpenAI response for {image_path}: {e}")
                                print(f"OpenAI Response Content: {message_content}")
                                return None
                        else:
                            print(f"Unexpected response format from OpenAI API for {image_path}.")
                            return None
                    else:
                        print(f"Request failed with status code {status} for {image_path}.")
                        # print(f"Response Content: {response_text}")
                        return None
        except aiohttp.ClientError as e:
            print(f"Request failed due to a client error: {e}")
            await asyncio.sleep(2 ** attempt)
        except Exception as e:
            print(f"Unexpected error occurred: {e}")
            await asyncio.sleep(2 ** attempt)
    return None


### 6) Scene Processing Functions

In [60]:
async def process_scenes_output(output_dir, json_output_dir):
    os.makedirs(json_output_dir, exist_ok=True)
    scenes = sorted([f for f in os.listdir(output_dir) if f.endswith('.jpg')], key=extract_scene_number)
    total_scenes = len(scenes)
    with tqdm(total=total_scenes, desc="Processing Scenes", unit="scene") as pbar:
        tasks = [process_single_scene(i, scene, output_dir, json_output_dir, pbar) for i, scene in enumerate(scenes)]
        await asyncio.gather(*tasks)


async def process_single_scene(i, scene, output_dir, json_output_dir, pbar):
    async with semaphore:  # Limit concurrent execution
        scene_path = os.path.join(output_dir, scene)

        # Encode image in base64
        base64_image = await encode_image(scene_path)

        # Perform color analysis
        color_analysis_result = analyze_image_colors(scene_path)

        # Send image to OpenAI for further analysis
        openai_response = await send_image_to_openai(scene_path, base64_image)

        # Check if openai_response is valid (not None or empty)
        if not openai_response:
            print(f"Skipping {scene} due to invalid OpenAI response.")
            # Log the base64 image and the scene path for debugging purposes
            print(f"Scene Path: {scene_path}")
            print(f"Base64 Image Length: {len(base64_image)}")
            print(f"Base64 Image Sample: {base64_image[:100]}...")  # Show a sample of the base64 string
            pbar.update(1)
            return

        # Combine both results, and include the reference to the image file
        final_output = {
            "Image File": scene,
            "Image Analysis": {
                **color_analysis_result["Color Analysis"],
                **openai_response.get("Image Analysis", {})
            }
        }

        # Save the final output to a JSON file in the specified folder
        output_filename = f'scene_{i + 1}_analysis.json'
        output_path = os.path.join(json_output_dir, output_filename)

        try:
            async with aiofiles.open(output_path, 'w') as json_file:
                await json_file.write(json.dumps(final_output, indent=4))
                print(f"Saved analysis for scene {i + 1} as {output_filename}")
        except Exception as e:
            print(f"Failed to save analysis for scene {i + 1}: {e}")

        pbar.update(1)



def extract_scene_number(filename):
    match = re.search(r'\d+', filename)
    return int(match.group()) if match else -1


### 7) Main Function Execution

In [61]:
def process_videos_in_directory(directory_path, output_base_dir):
    video_files = [f for f in os.listdir(directory_path) if f.endswith(('.mp4', '.avi', '.mkv'))]

    if not video_files:
        print("No video files found in the directory.", flush=True)
        return

    with tqdm(total=len(video_files), desc="Processing Videos", unit="video") as pbar:
        for i, video_file in enumerate(video_files):
            start_time = time.time()

            video_path = os.path.join(directory_path, video_file)
            video_name = os.path.splitext(video_file)[0]
            video_size = os.path.getsize(video_path)

            video_output_dir = os.path.join(output_base_dir, video_name)
            scenes_output_dir = os.path.join(video_output_dir, 'scenes_output')
            json_output_dir = os.path.join(video_output_dir, 'json_output')

            os.makedirs(scenes_output_dir, exist_ok=True)

            print(f"Processing video {i + 1}/{len(video_files)}: {video_file}", flush=True)
            
            scenes = analyze_video(video_path)
            extract_frames_imageio(video_path, scenes, scenes_output_dir)
            asyncio.run(process_scenes_output(scenes_output_dir, json_output_dir))  # Run async scene processing

            end_time = time.time()
            processing_time = end_time - start_time

            summary = {
                "Video Title": video_file,
                "File Size (bytes)": video_size,
                "Processing Time (seconds)": processing_time
            }

            summary_output_path = os.path.join(output_base_dir, f"{video_name}_summary.json")
            with open(summary_output_path, 'w') as summary_file:
                json.dump(summary, summary_file, indent=4)

            print(f"Finished processing video: {video_file}", flush=True)
            pbar.update(1)


# Main script execution
video_directory = '/Users/santiagowon/Dropbox/Santiago/01. Maestria/Tesis/02_Video_DB'
output_base_directory = '/Users/santiagowon/Dropbox/Santiago/01. Maestria/Tesis/11_Project_Analysed_DB'

process_videos_in_directory(video_directory, output_base_directory)
print("FINISHED PROCESSING ALL VIDEOS.", flush=True)


Processing Videos:   0%|          | 0/27 [00:00<?, ?video/s]

Processing video 1/27: Avatar_The_Last_Airbender.mp4


ERROR:pyscenedetect:VideoManager is deprecated and will be removed.
INFO:pyscenedetect:Loaded 1 video, framerate: 25.000 FPS, resolution: 626 x 480
INFO:pyscenedetect:Downscale factor set to 2, effective resolution: 313 x 240
INFO:pyscenedetect:Detecting scenes...
INFO:root:Detected 338 scenes:
INFO:root:Scene 1: Start 00:00:00.000 / Frame 0, End 00:00:02.520 / Frame 63
INFO:root:Scene 2: Start 00:00:02.520 / Frame 63, End 00:00:04.360 / Frame 109
INFO:root:Scene 3: Start 00:00:04.360 / Frame 109, End 00:00:06.440 / Frame 161
INFO:root:Scene 4: Start 00:00:06.440 / Frame 161, End 00:00:08.200 / Frame 205
INFO:root:Scene 5: Start 00:00:08.200 / Frame 205, End 00:00:16.520 / Frame 413
INFO:root:Scene 6: Start 00:00:16.520 / Frame 413, End 00:00:22.960 / Frame 574
INFO:root:Scene 7: Start 00:00:22.960 / Frame 574, End 00:00:29.800 / Frame 745
INFO:root:Scene 8: Start 00:00:29.800 / Frame 745, End 00:00:33.200 / Frame 830
INFO:root:Scene 9: Start 00:00:33.200 / Frame 830, End 00:00:53.360 

Extracted and saved middle frame of scene 1 as /Users/santiagowon/Dropbox/Santiago/01. Maestria/Tesis/11_Project_Analysed_DB/Avatar_The_Last_Airbender/scenes_output/scene_1.jpg
Extracted and saved middle frame of scene 2 as /Users/santiagowon/Dropbox/Santiago/01. Maestria/Tesis/11_Project_Analysed_DB/Avatar_The_Last_Airbender/scenes_output/scene_2.jpg
Extracted and saved middle frame of scene 3 as /Users/santiagowon/Dropbox/Santiago/01. Maestria/Tesis/11_Project_Analysed_DB/Avatar_The_Last_Airbender/scenes_output/scene_3.jpg
Extracted and saved middle frame of scene 4 as /Users/santiagowon/Dropbox/Santiago/01. Maestria/Tesis/11_Project_Analysed_DB/Avatar_The_Last_Airbender/scenes_output/scene_4.jpg
Extracted and saved middle frame of scene 5 as /Users/santiagowon/Dropbox/Santiago/01. Maestria/Tesis/11_Project_Analysed_DB/Avatar_The_Last_Airbender/scenes_output/scene_5.jpg
Extracted and saved middle frame of scene 6 as /Users/santiagowon/Dropbox/Santiago/01. Maestria/Tesis/11_Project_An

Response Status Code: 200
Response Content: {
  "id": "chatcmpl-A1zeBZRM6M55uT6LSMZHzL4DZFKR7",
  "object": "chat.completion",
  "created": 1725038663,
  "model": "gpt-4o-2024-05-13",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "{\n    \"ImageAnalysis\": {\n        \"Suitability\": {\n            \"General_Audiences\": true,\n            \"Age_Restriction\": \"None\"\n        },\n        \"Objects\": [\n            {\n                \"Type\": \"Text\",\n                \"Language\": \"Chinese\",\n                \"Content\": \"火烈\"\n            },\n            {\n                \"Type\": \"Logo\",\n                \"Content\": \"Nickelodeon Sonic\",\n                \"Position\": \"Top right corner\"\n            },\n            {\n                \"Type\": \"Animation\",\n                \"Description\": \"A character surrounded by flames\",\n                \"Position\": \"Bottom right\"\n            }\n        ],\n     

...

Response Status Code: 200
Response Content: {
  "id": "chatcmpl-A1zfAD6PctLeHUASwWg9UPGFAUC2K",
  "object": "chat.completion",
  "created": 1725038724,
  "model": "gpt-4o-2024-05-13",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "{\n  \"ImageAnalysis\": {\n    \"Suitability\": {\n      \"GeneralAudience\": true,\n      \"InappropriateContent\": false\n    },\n    \"Objects\": [\n      {\n        \"Type\": \"MusicalInstrument\",\n        \"Name\": \"Guitar-like\",\n        \"Color\": \"Red\"\n      },\n      {\n        \"Type\": \"Clothing\",\n        \"Name\": \"Robe\",\n        \"Color\": \"Blue and Yellow\"\n      },\n      {\n        \"Type\": \"Accessory\",\n        \"Name\": \"Necklace\",\n        \"Color\": \"White\"\n      },\n      {\n        \"Type\": \"Accessory\",\n        \"Name\": \"Bracelets\",\n        \"Color\": \"White\"\n      }\n    ],\n    \"Place\": {\n      \"Setting\": \"Forest\",\n      \"Vegetation\":

Response Status Code: 200
Response Content: {
  "id": "chatcmpl-A1zfAoggSfEfSRy1Iy703h9P0bQjU",
  "object": "chat.completion",
  "created": 1725038724,
  "model": "gpt-4o-2024-05-13",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "{\n  \"Image Analysis\": {\n    \"Suitability\": \"Suitable for general audiences\",\n    \"Objects\": [\n      {\n        \"type\": \"rock formation\",\n        \"description\": \"Large rock cliffs on both sides of the image suggesting a canyon setting.\"\n      },\n      {\n        \"type\": \"trees\",\n        \"description\": \"A variety of green trees lining the cliffs and scattered in the background.\"\n      },\n      {\n        \"type\": \"water\",\n        \"description\": \"A body of water in the foreground with some rocks partially submerged.\"\n      },\n      {\n        \"type\": \"animals\",\n        \"description\": \"Large insect-like creatures partially submerged in the water.\"\n   

Response Status Code: 200
Response Content: {
  "id": "chatcmpl-A1zfCjMpS0Cqcip9rwR1hQ6MUcVWy",
  "object": "chat.completion",
  "created": 1725038726,
  "model": "gpt-4o-2024-05-13",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "{\n  \"Image Analysis\": {\n    \"Suitability\": \"Suitable for general audiences\",\n    \"Objects\": [\n      {\n        \"Type\": \"Character\",\n        \"Description\": \"Animated character with a hat and light-colored clothing sitting cross-legged on the ground, appearing to hold an object\"\n      },\n      {\n        \"Type\": \"Character\",\n        \"Description\": \"Animated character in colorful clothing standing, playing a musical instrument\"\n      },\n      {\n        \"Type\": \"Character\",\n        \"Description\": \"Animated character with a headpiece, sitting cross-legged, and holding an object up\"\n      }\n    ],\n    \"Place\": {\n      \"Setting\": \"Outdoor forest scene wit

Processing Videos:   0%|          | 0/27 [02:44<?, ?video/s]


KeyboardInterrupt: 